In [1]:
import os, typing, json

from conflux_web3 import Web3
from conflux_web3.dev import get_testnet_web3
from dotenv import load_dotenv

load_dotenv()

class Metadata(typing.TypedDict):
    abi: typing.Any
    bytecode: typing.Any

def get_contract_metadata(path_: str) -> Metadata:
    with open(path_) as f:
        metadata = json.load(f)
        return {
            "abi": metadata["abi"],
            "bytecode": metadata["bytecode"]["object"]
        }

In [9]:
w3 = get_testnet_web3()
acct = w3.account.from_key(os.environ["SK"])
w3.cfx.default_account = acct

Registry = w3.cfx.contract(**get_contract_metadata("./out/ERC6551Registry.sol/ERC6551Registry.json"))
registry_address = Registry.constructor().transact().executed()["contractCreated"]
if registry_address is None:
    raise Exception("unexpected result")
print(f"registry address: {registry_address}")
registry = Registry(
    registry_address
)

registry address: cfx:ach1zpmjyv17arzdjx9fgbt4cw6uns0ntuwre8sdsy


In [10]:
Implementation = w3.cfx.contract(**get_contract_metadata("./out/ERC6551AccountUpgradeable.sol/ERC6551AccountUpgradeable.json"))
simple_impl_addr = Implementation.constructor().transact().executed()["contractCreated"]
if simple_impl_addr is None:
    raise Exception("unexpected result")
# print(f"implementation result: {simple_impl_addr}")
# acct_impl = Implementation(simple_impl_addr)
simple_impl_addr

'cfx:acd9jgz4d5as156ef08r5mdgux28y51yz639nr5s78'

In [11]:
admin_control = w3.cfx.contract(name="AdminControl", with_deployment_info=True)
admin_control.functions.setAdmin(registry.address, w3.address.zero_address()).transact().executed()
admin_control.functions.setAdmin(simple_impl_addr, w3.address.zero_address()).transact().executed()


AttributeDict({'transactionHash': HexBytes('0x104fbd92baf35dc1825413a470f80429ad58dd09a0a0718c9ac07565438abb7c'),
 'index': 0,
 'blockHash': HexBytes('0xae0900e5691ed42443bce55b5c3bcad2fe522d7b78d0a8faa5876f6c6deded18'),
 'epochNumber': 78314716,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:aaejuaaaaaaaaaaaaaaaaaaaaaaaaaaaaa2mhjju8k',
 'gasUsed': 27808,
 'gasFee': 27808000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'sta

In [4]:
# Below code are used under testnet development

nft_contract_address = (
    "CFXTEST:..."
)
nft_contract = w3.cfx.contract(
    address=nft_contract_address, **get_contract_metadata("./out/IERC721.sol/IERC721.json")
)
token_id = ...

tba_acct_addr = registry.functions.account(
    simple_impl_addr, w3.cfx.chain_id, nft_contract_address, token_id, 0
).call()

if not (code := w3.cfx.get_code(tba_acct_addr)):
    registry.functions.createAccount(
        simple_impl_addr, w3.cfx.chain_id, nft_contract_address, token_id, 0, "0x"
    ).transact().executed()

tba = Implementation(tba_acct_addr)

In [5]:
tba.address

'CFXTEST:TYPE.CONTRACT:ACC5XK7B7JAZCRRBFEH1RHPGNY59BZ5D8PP0YPK630'

In [6]:
owner_acct = w3.cfx.account.from_key(os.environ["OWNER_SK"])
w3.wallet.add_account(owner_acct)

In [7]:
w3.cfx._get_admin(tba.address, None)

'cfxtest:aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa6f0vrcsw'

In [8]:
w3.cfx.get_code(tba.address)

HexBytes('0x363d3d373d3d3d363d738498075dc27b3bff88e5584a517f546ceac31bf65af43d82803e903d91602b57fd5bf3000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000008c8ebd56597d2904a3298ca8562bed3bb6e75d2a0000000000000000000000000000000000000000000000000000126641de1e81')

In [ ]:
w3.cfx.send_transaction({
    "to": tba.address,
    "value": 20 * 10**18
}).executed()
tba.functions.execute(w3.cfx.default_account, 10*10**18, "0x", 0).transact({"from": owner_acct.address}).executed()


In [12]:
w3.cfx.get_balance(tba.address).to("CFX")

10 CFX

In [14]:
usdt = w3.cfx.contract(name="cUSDT")

In [15]:
usdt.functions.transfer(tba.address, 10**19).transact().executed()

AttributeDict({'transactionHash': HexBytes('0xe99c6edc0ebb8161dbb903ce54fc62fc650f948875c830b799f1e596ff00cea8'),
 'index': 0,
 'blockHash': HexBytes('0x18d10c2c0a00e6fc79f1e32c313cc3ea88ca46998ad5d2da6d0b5ecf9c610302'),
 'epochNumber': 136100454,
 'from': 'cfxtest:aanthezva9vuyrpsmb15sf9xt97f8vduky7cebydev',
 'to': 'cfxtest:acepe88unk7fvs18436178up33hb4zkuf62a9dk1gv',
 'gasUsed': 61905,
 'gasFee': 61905000000000 Drip,
 'contractCreated': None,
 'logs': [AttributeDict({'address': 'cfxtest:acepe88unk7fvs18436178up33hb4zkuf62a9dk1gv',
   'topics': [HexBytes('0x06b541ddaa720db2b10a4d0cdac39b8d360425fc073085fac19bc82614677987'),
    HexBytes('0x00000000000000000000000016f392b107e30a358e506fb717f37ffa5f44704d'),
    HexBytes('0x00000000000000000000000016f392b107e30a358e506fb717f37ffa5f44704d'),
    HexBytes('0x00000000000000000000000085b9a7a1ea015135a1290f769d865d37f0d763f3')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000008ac7230489e80000000000000000000000000000000

In [26]:
data = usdt.functions.transfer(w3.cfx.default_account, 10**18).build_transaction({"from": w3.cfx.default_account})["data"]
tba.functions.execute(usdt.address, 0, data, 0).transact({"from": owner_acct.address}).executed()

AttributeDict({'transactionHash': HexBytes('0xa530640d2b82309ad5b3efb666027d362d782c5c4c94c60802096f94fecc48cf'),
 'index': 0,
 'blockHash': HexBytes('0x5b37cd4864b88035ce66c03d04d29686b469347c52dc06f06a0e3719d6965d53'),
 'epochNumber': 136100985,
 'from': 'cfxtest:aanhtnrex2nj56kkbws4yx0jeab34ae16pcap53w13',
 'to': 'cfxtest:acc5xk7b7jazcrrbfeh1rhpgny59bz5d8pp0ypk630',
 'gasUsed': 60827,
 'gasFee': 60827000000000 Drip,
 'contractCreated': None,
 'logs': [AttributeDict({'address': 'cfxtest:acepe88unk7fvs18436178up33hb4zkuf62a9dk1gv',
   'topics': [HexBytes('0x06b541ddaa720db2b10a4d0cdac39b8d360425fc073085fac19bc82614677987'),
    HexBytes('0x00000000000000000000000085b9a7a1ea015135a1290f769d865d37f0d763f3'),
    HexBytes('0x00000000000000000000000085b9a7a1ea015135a1290f769d865d37f0d763f3'),
    HexBytes('0x00000000000000000000000016f392b107e30a358e506fb717f37ffa5f44704d')],
   'data': HexBytes('0x0000000000000000000000000000000000000000000000000de0b6b3a7640000000000000000000000000000000

In [27]:
usdt.functions.balanceOf(tba.address).call()

9000000000000000000